In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import matplotlib.colors
import statsmodels.formula.api as smf
import seaborn as sns
from cem.match import match
from cem.coarsen import coarsen
from cem.imbalance import L1
import statsmodels.api as sm


In [2]:
matplotlib.rc('font', family='Arial') 

In [129]:

pop_df=pd.read_csv(r'C:\Users\Yasaman\Downloads\World_bank_population.csv',skiprows=3)
pop_df['Country Code']=pop_df['Country Code'].apply(lambda x: x.lower())
possible_countries=pop_df.query(" `2019` >=1000000")['Country Code'].values

excluded_iso3_codes = [
    "IRL",  # Ireland
    "SSD",  # South Sudan
    "SDN",  # Sudan
    "COG",  # Republic of the Congo
    "COD",  # Democratic Republic of the Congo
    "GIN",  # Guinea
    "GNB",  # Guinea-Bissau
    "GNQ",  # Equatorial Guinea
    "PNG",  # Papua New Guinea
    "XKX",  # Kosovo (unofficial)
    "MNE",  # Montenegro
    "SRB",  # Serbia
    "TLS",   # Timor-Leste
    "GEO", #Georgia
    'SWZ', 
    'PRK', #North Korea
]
excluded_iso3_codes=[c.lower() for c in excluded_iso3_codes]


possible_iso=list(set(possible_countries)-set(excluded_iso3_codes))

In [201]:

df = pd.read_csv(r"C:\Users\Yasaman\Downloads\Attention-fractional counting.csv")
df=df[df['country'].isin(possible_iso)]
df.rename(columns={'aggregated_value': 'count', 'country': 'Mention_country', 'affiliation_country': 'Aff_country'}, inplace=True)
df = df[df['year'].isin(np.arange(2002, 2020))]
Country_list={'Egypt':'EGY', 'Tunisia':'TUN','Libya':'LBY','Syria':'SYR','Yemen':'YEM','Bahrain':'BHR','Jordan':'JOR','Kuwait':'KWT','Morocco':'MAR','Oman':'OMN'}
rev_Country_list={Country_list[key]: key for key in Country_list}
abbr=[country.lower() for country in Country_list.values()]
physical_sciences=['MATH', 'ENGI', 'PHYS', 'COMP', 'MUL']
df=df[~df['subjarea'].isin(physical_sciences)]

df=df.groupby(['year', 'Mention_country'])['count'].sum().reset_index()


In [202]:
data=pd.read_csv(r"C:\Users\Yasaman\Downloads\scopus_2024_V1_scholarlymigration_country_enriched.csv")
data=data[data['year'].isin(np.arange(2002, 2020))]
data=data[['iso3code', 'incomelevel', 'gdp_per_capita', 'year', 'population', 'region']].dropna()
data.rename(columns={'iso3code':'Mention_country'}, inplace=True)
data['Mention_country']=data['Mention_country'].apply(lambda x: x.lower())
df=df.merge(data, on=['Mention_country', 'year'], how='outer')

In [204]:
# Define the required year range
required_years = list(range(2002, 2020))

# Get the unique countries
unique_countries = df["Mention_country"].unique()

# Create a complete DataFrame with all country-year combinations
full_data = []
for country in unique_countries:
    country_data = df[df["Mention_country"] == country]
    existing_years = set(country_data["year"])
    
    for year in required_years:
        if year in existing_years:
            row = country_data[country_data["year"] == year].iloc[0].to_dict()
        else:
            row = {
                "year": year,
                "Mention_country": country,
                "count": 0,
                "incomelevel": country_data["incomelevel"].iloc[0] if not country_data.empty else np.nan,
                "gdp_per_capita": np.nan,
                "population": np.nan,
                "region": country_data["region"].iloc[0] if not country_data.empty else np.nan,
            }
        full_data.append(row)

# Convert to DataFrame
df_complete = pd.DataFrame(full_data)


In [241]:
df_complete['treated']=df_complete['Mention_country'].isin(abbr).astype(int)
df_complete['post']=df_complete['year'].apply(lambda x: 0 if x>=2002 and x<=2010 else 1 )
df_complete['count']=df_complete['count'].fillna(0)
df_complete['log_count']=np.log10(df_complete['count']+1)
df_complete[['incomelevel', 'region', 'gdp_per_capita', 'population']] = df_complete.groupby('Mention_country')[['incomelevel', 'region', 'gdp_per_capita', 'population']].ffill()
df_complete[['incomelevel', 'region', 'gdp_per_capita', 'population']] = df_complete.groupby('Mention_country')[['incomelevel', 'region', 'gdp_per_capita', 'population']].bfill()
df_complete['log_gdp']=np.log10(df_complete['gdp_per_capita'])
df_complete['log_population']=np.log10(df_complete['population'])
df_complete[df_complete['Mention_country'].isin(possible_iso)].reset_index(drop=True)

,year,Mention_country,count,incomelevel,gdp_per_capita,population,region,treated,post,log_count,log_gdp,log_population
0,2002.0,afg,53.925109,LIC,182.174037,21000256.0,South Asia,0,0,1.739771,2.260486,7.322225
1,2003.0,afg,62.787674,LIC,199.643228,22645130.0,South Asia,0,0,1.804737,2.300255,7.354975
2,2004.0,afg,72.018290,LIC,221.830531,23553551.0,South Asia,0,0,1.863432,2.346021,7.372056
3,2005.0,afg,79.680195,LIC,254.115274,24411191.0,South Asia,0,0,1.906767,2.405031,7.387589
4,2006.0,afg,84.109066,LIC,274.015394,25442944.0,South Asia,0,0,1.929976,2.437775,7.405567
...,...,...,...,...,...,...,...,...,...,...,...,...
2605,2015.0,zwe,416.835292,LMC,1410.329173,14154937.0,Sub-Saharan Africa,0,1,2.621005,3.149320,7.150908
2606,2016.0,zwe,474.749376,LMC,1421.787791,14452704.0,Sub-Saharan Africa,0,1,2.677378,3.152835,7.159949
2607,2017.0,zwe,526.176430,LMC,1192.107012,14751101.0,Sub-Saharan Africa,0,1,2.721956,3.076315,7.168824
2608,2018.0,zwe,599.204753,LMC,2269.177012,15052184.0,Sub-Saharan Africa,0,1,2.778299,3.355868,7.177600


In [257]:
# Running the Difference-in-Differences regression
model = smf.ols("log_count ~ treated * post + log_gdp   + C(Mention_country) + C(year)", data=df_complete).fit()
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_count   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.991
Method:                 Least Squares   F-statistic:                     1850.
Date:                Fri, 07 Mar 2025   Prob (F-statistic):               0.00
Time:                        11:43:31   Log-Likelihood:                 2658.6
No. Observations:                3762   AIC:                            -4861.
Df Residuals:                    3534   BIC:                            -3440.
Df Model:                         227                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [263]:
# Running the Difference-in-Differences regression
model = smf.ols("log_count ~ treated * post + log_gdp + C(Mention_country) + C(year)", data=df_complete[df_complete['region']=='Middle East & North Africa']).fit()
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_count   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     426.4
Date:                Fri, 07 Mar 2025   Prob (F-statistic):          4.43e-263
Time:                        11:47:02   Log-Likelihood:                 308.82
No. Observations:                 378   AIC:                            -537.6
Df Residuals:                     338   BIC:                            -380.2
Df Model:                          39                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [260]:
# Running the Difference-in-Differences regression
model = smf.ols("log_count ~ treated * post  + C(Mention_country) + C(year)", data=df_complete[df_complete['region']=='Middle East & North Africa']).fit()
# Print summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:              log_count   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     438.2
Date:                Fri, 07 Mar 2025   Prob (F-statistic):          2.67e-264
Time:                        11:44:41   Log-Likelihood:                 308.55
No. Observations:                 378   AIC:                            -539.1
Df Residuals:                     339   BIC:                            -385.6
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept             

In [262]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
df_complete['treated:post']=df_complete['treated']*df_complete['post']
# Select the independent variables (excluding categorical variables handled by C())
independent_vars = df_complete[['treated', 'post', 'treated:post', 'log_gdp', 'log_population']]

# Compute VIF
vif_data = pd.DataFrame()
vif_data["Variable"] = independent_vars.columns
vif_data["VIF"] = [variance_inflation_factor(independent_vars.values, i) for i in range(len(independent_vars.columns))]

print(vif_data)


         Variable        VIF
0         treated   2.107602
1            post   2.154050
2    treated:post   2.101720
3         log_gdp  14.662263
4  log_population  14.223643


In [266]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_complete["log_gdp_scaled"] = scaler.fit_transform(df_complete[["log_gdp"]])

model_mixed = smf.mixedlm(
    "log_count ~ treated * post + log_gdp_scaled + C(year)", 
    data=df_complete,
    groups=df_complete["Mention_country"]
).fit(reml=True)

print(model_mixed.summary())


c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(


                 Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      log_count
No. Observations:       3762         Method:                  REML     
No. Groups:             209          Scale:                   0.0152   
Min. group size:        18           Log-Likelihood:          1695.7592
Max. group size:        18           Converged:               No       
Mean group size:        18.0                                           
-----------------------------------------------------------------------
                  Coef.   Std.Err.    z    P>|z|    [0.025     0.975]  
-----------------------------------------------------------------------
Intercept          1.590      0.090 17.703 0.000       1.414      1.766
C(year)[T.2003.0] -0.001      0.012 -0.056 0.955      -0.024      0.023
C(year)[T.2004.0] -0.001      0.012 -0.075 0.940      -0.025      0.023
C(year)[T.2005.0]  0.012      0.012  0.951 0.341      -0.013      0.036
C(year)[T

c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\Yasaman\anaconda3\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 1.687602
  warnings.warn(msg, ConvergenceWarning)
